In [1]:
import warnings
import logging
# warnings.filterwarnings("ignore")

In [20]:
import datetime
import glob
from functools import lru_cache, partial
from pprint import pprint
import ray

import gym_trading_env
import gymnasium as gym
import matplotlib.pyplot as plt
import MultiTrade
import numpy as np
import pandas as pd
import copy
import torch
from gym_trading_env.downloader import download
from gym_trading_env.environments import TradingEnv
from gym_trading_env.renderer import Renderer
from IPython.display import display
from ray import train, tune
from tqdm.autonotebook import tqdm
from utils.utils import build_dataset, build_market_image,preprocess_data,stack_arrays

from utils.utils import build_dataset, build_market_image
from MultiTrade.environments import ForecastingTradingEnv,NeuralForecastingTradingEnv
import warnings
warnings.filterwarnings('ignore')
warnings.simplefilter('ignore')

from neuralforecast.core import NeuralForecast
from ray.tune.schedulers import HyperBandScheduler
from ray.tune import ProgressReporter,JupyterNotebookReporter
from ray.tune.search import ConcurrencyLimiter
from ray.tune.search.optuna import OptunaSearch 
from ray.tune.search.hyperopt import HyperOptSearch

from ray.rllib.algorithms.ppo import PPOConfig
from ray.rllib.algorithms.appo import APPOConfig
from ray.rllib.algorithms.dreamerv3 import DreamerV3Config
from ray.tune.logger import pretty_print
from ray.rllib.algorithms.algorithm import Algorithm
from ray.rllib.algorithms.algorithm_config import AlgorithmConfig
from ray.rllib.evaluation.worker_set import WorkerSet
from ray.rllib.evaluation.metrics import summarize_episodes
from ray.rllib.env import BaseEnv
from ray.rllib.policy import Policy
from ray.rllib.evaluation import Episode, RolloutWorker
from ray.rllib.algorithms.callbacks import DefaultCallbacks
from typing import Dict, Tuple
from ray.tune.registry import register_env
from ray.rllib.algorithms import ppo

In [21]:
%%writefile custom.css
/* custom.css */

/* Set background color for all widgets */
.tuneStatus {
    /* white background */
    background-color: #ffffff;
    
    width: 100%;
    height: 100%;
    overflow:'auto',
    transition:'all'
}

Overwriting custom.css


In [22]:
from IPython.display import display, HTML
import tensorboard

# Load custom CSS file
css = HTML('<link rel="stylesheet" type="text/css" href="./custom.css">') # make sure the path is correct, it fails silently otherwise
display(css)

In [23]:
COIN_PAIRS=['BTC/USDT','ETH/USDT','SOL/USDT','BNB/USDT','XRP/USDT','ADA/USDT',
            'ETH/BTC','SOL/ETH','BNB/ETH','XRP/ETH',"ADA/ETH",
            'SOL/BTC','SOL/BNB',
            'XRP/BTC','XRP/BNB',
            'ADA/BTC','ADA/BNB',
            ]
target_pair='ETHUSDT'
time_frame="1h"

In [24]:

# download(exchange_names = ["binance"],
#     symbols= tqdm(COIN_PAIRS),
#     timeframe= time_frame,
#     dir = "data",
#     since= datetime.datetime(year= 2022, month= 1, day=1),
# )

In [25]:
model=NeuralForecast.load("C:/Users/standard/Git/MultiTrader/forecaster_pretrain/neuralForecast")

Seed set to 1
Seed set to 1
Seed set to 1


In [26]:
import warnings
warnings.filterwarnings('ignore')

In [27]:

symbol='ETH/USDT'
data=build_market_image(target_pair=symbol,time_frame='1h',axis=0)
data=data[data['symbol']==target_pair.replace('/','')].copy()
data['ds']=data.index



In [28]:
split_date=datetime.datetime(year= 2024, month= 3, day=1)
end_date=split_date+datetime.timedelta(days=14)

symbol='ETHUSDT'
hf_data=data.copy()

hf_train_data=hf_data.groupby('symbol').apply(lambda x: x[:split_date])
hf_test_data=hf_data.groupby('symbol').apply(lambda x: x[split_date:end_date])

hf_train_data=hf_train_data.reset_index(level=0,drop=True).reset_index()
hf_test_data=hf_test_data.reset_index(level=0,drop=True).reset_index()


trade_data=data[data['symbol']==symbol].copy()
train_data=trade_data[:split_date]
test_data=trade_data[split_date:end_date]
# hf_test_data

In [29]:
train_data.head(5)

,open,high,low,close,volume,y,feature_hour_of_day,feature_day_of_week,feature_day_of_month,feature_day_of_year,...,feature_log_volume_15,feature_log_return_20,feature_log_volume_20,feature_log_return_25,feature_log_volume_25,feature_log_return_30,feature_log_volume_30,unique_id,symbol,ds
date_open,,,,,,,,,,,,,,,,,,,,,
2022-01-01 06:00:00,2831.117702,2814.083234,2826.553006,2807.447705,5239.0026,2807.447705,1.260870,1.833333,1.0,1.00551,...,43.570511,1.831037,43.934894,1.526594,43.909164,1.96648,42.248218,8,ETHUSDT,2022-01-01 06:00:00
2022-01-01 07:00:00,2807.447702,2796.343234,2819.223006,2810.837705,6541.6499,2810.837705,1.304348,1.833333,1.0,1.00551,...,43.570511,1.831037,43.934894,1.526594,43.909164,1.96648,42.248218,8,ETHUSDT,2022-01-01 07:00:00
2022-01-01 08:00:00,2810.837702,2805.253234,2826.823006,2816.927705,4013.6403,2816.927705,1.347826,1.833333,1.0,1.00551,...,43.570511,1.831037,43.934894,1.526594,43.909164,1.96648,42.248218,8,ETHUSDT,2022-01-01 08:00:00
2022-01-01 09:00:00,2816.657702,2801.413234,2827.603006,2814.197705,4515.6636,2814.197705,1.391304,1.833333,1.0,1.00551,...,43.570511,1.831037,43.934894,1.526594,43.909164,1.96648,42.248218,8,ETHUSDT,2022-01-01 09:00:00
2022-01-01 10:00:00,2814.187702,2803.293234,2791.683006,2786.917705,10412.4737,2786.917705,1.434783,1.833333,1.0,1.00551,...,43.570511,1.831037,43.934894,1.526594,43.909164,1.96648,42.248218,8,ETHUSDT,2022-01-01 10:00:00


In [30]:
def prep_forecasts(df):
    forecast_array=[]
    # print(self.df.columns)

    model.dataset, model.uids, model.last_dates, model.ds=model._prepare_fit(df[['ds','unique_id','y']],
                                                                                        static_df=None, 
                                                                                        sort_df=None,
                                                                                        predict_only=False,
                                                                                        id_col='unique_id', 
                                                                                        time_col='ds', 
                                                                                        target_col='y')
    forecasts=model.predict_insample()
    forecasts_series=forecasts.groupby('cutoff').apply(lambda x: x.select_dtypes(np.number).values.flatten())
    new_df=df[df['ds'].isin([c for c in forecasts_series.index])]
    forecasts_series=forecasts_series[new_df.index]
    forecast_array=[c for c in forecasts_series]
    return forecast_array,new_df

In [31]:
train_forecast_array,train_df=prep_forecasts(train_data)
test_forecast_array,test_df=prep_forecasts(test_data)

In [32]:
def rew_func(history):
    portfolio_initial_value= history["portfolio_valuation",0]
    current_position=history['position',-1]
    portfolio_return=history["portfolio_valuation"]
    portfolio_pct_change=pd.Series(np.array(portfolio_return)).pct_change().fillna(0)
    # print(portfolio_pct_change)
    reward = (portfolio_pct_change.mean() / portfolio_pct_change.std())
    
    # print(np.isnan(reward),reward)
    reward = 0 if np.isnan(reward) else reward
    # print(np.isnan(reward),reward)
    return float(reward)


In [33]:
train_env_config=dict(
                name='ETHUSDT_train',
                forecast_horizon=7,
                context_length=35,
                reward_function=rew_func,
                forecasts=train_forecast_array,
                df = train_df, # Your dataset with your custom features
                positions = [ -.5,-.25,.25, .5], # -1 (=SHORT), 0(=OUT), +1 (=LONG)
                trading_fees = 0.01/100, # 0.01% per stock buy / sell (Binance fees)
                borrow_interest_rate= 0.0003/100, # 0.0003% per timestep (one timestep = 1h here)
                max_episode_duration=168,
                verbose=0
                )

test_env_config=copy.deepcopy(train_env_config)
test_env_config['name']='ETHUSDT_test'
test_env_config['df'] = test_df
test_env_config['forecasts']=test_forecast_array



In [16]:
train_env=NeuralForecastingTradingEnv(**train_env_config)

In [34]:
obs,info=train_env.reset()
obs.shape

(51,)

In [35]:
train_env.df

,open,high,low,close,volume,y,feature_hour_of_day,feature_day_of_week,feature_day_of_month,feature_day_of_year,...,feature_log_volume_20,feature_log_return_25,feature_log_volume_25,feature_log_return_30,feature_log_volume_30,unique_id,symbol,ds,dynamic_feature__0,dynamic_feature__1
date_open,,,,,,,,,,,,,,,,,,,,,
2022-01-01 06:00:00,2831.117702,2814.083234,2826.553006,2807.447705,5239.0026,2807.447705,1.260870,1.833333,1.000000,1.005510,...,43.934894,1.526594,43.909164,1.966480,42.248218,8,ETHUSDT,2022-01-01 06:00:00,0,0
2022-01-01 07:00:00,2807.447702,2796.343234,2819.223006,2810.837705,6541.6499,2810.837705,1.304348,1.833333,1.000000,1.005510,...,43.934894,1.526594,43.909164,1.966480,42.248218,8,ETHUSDT,2022-01-01 07:00:00,0,0
2022-01-01 08:00:00,2810.837702,2805.253234,2826.823006,2816.927705,4013.6403,2816.927705,1.347826,1.833333,1.000000,1.005510,...,43.934894,1.526594,43.909164,1.966480,42.248218,8,ETHUSDT,2022-01-01 08:00:00,0,0
2022-01-01 09:00:00,2816.657702,2801.413234,2827.603006,2814.197705,4515.6636,2814.197705,1.391304,1.833333,1.000000,1.005510,...,43.934894,1.526594,43.909164,1.966480,42.248218,8,ETHUSDT,2022-01-01 09:00:00,0,0
2022-01-01 10:00:00,2814.187702,2803.293234,2791.683006,2786.917705,10412.4737,2786.917705,1.434783,1.833333,1.000000,1.005510,...,43.934894,1.526594,43.909164,1.966480,42.248218,8,ETHUSDT,2022-01-01 10:00:00,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2024-02-29 13:00:00,2560.127702,2563.143234,2564.693006,2574.107705,27353.9760,2574.107705,1.565217,1.500000,1.933333,1.167153,...,41.948350,1.575298,43.749575,2.018276,42.143923,8,ETHUSDT,2024-02-29 13:00:00,0,0
2024-02-29 14:00:00,2574.097702,2568.653234,2569.123006,2564.577705,33678.8986,2564.577705,1.608696,1.500000,1.933333,1.167153,...,42.874410,1.560110,43.519540,2.007139,41.877627,8,ETHUSDT,2024-02-29 14:00:00,0,0
2024-02-29 15:00:00,2564.567702,2557.073234,2574.183006,2563.947705,22287.8112,2563.947705,1.652174,1.500000,1.933333,1.167153,...,43.429008,1.559725,43.174088,1.998366,41.457440,8,ETHUSDT,2024-02-29 15:00:00,0,0


In [17]:
obs=train_env.reset()
terminated = truncated = False
# env.action_space.sa
while not terminated and not truncated:
    action = train_env.action_space.sample()
    obs, reward, terminated, truncated, info = train_env.step(action)
info['reward']
# train_env.results_metrics
# train_env.historical_info.columns

-0.009705888021481562

In [18]:


register_env("train_env", lambda config: NeuralForecastingTradingEnv(**train_env_config))
register_env("test_env", lambda config: NeuralForecastingTradingEnv(**test_env_config))


In [ ]:

def trial_str_creator(trial):
    return "{}_{}_trading_agent".format(trial.trainable_name, trial.trial_id)



In [ ]:
# type(float(np.float64(1660)))

In [ ]:
class TraderCallbacks(DefaultCallbacks):
    
    def on_episode_start(
            self,
            *,
            worker: RolloutWorker,
            base_env: BaseEnv,
            policies: Dict[str, Policy],
            episode: Episode,
            env_index: int,
            **kwargs,
        ):
        # Make sure this episode has just been started (only initial obs
        # logged so far).


        # Create lists to store angles in
        episode.hist_data["portfolio_values"] = []
        episode.hist_data["positions"] = []

    def on_episode_step(
        self,
        *,
        worker: RolloutWorker,
        base_env: BaseEnv,
        policies: Dict[str, Policy],
        episode: Episode,
        env_index: int,
        **kwargs,
    ):
        # Make sure this episode is ongoing.

        portfolio_valuation = episode.last_info_for()['portfolio_valuation']
        position=episode.last_info_for()['position']
        episode.hist_data["portfolio_values"].append(portfolio_valuation)
        episode.hist_data["positions"].append(position)

        
    def on_episode_end(
        self,
        *,
        worker: RolloutWorker,
        
        base_env: BaseEnv,
        policies: Dict[str, Policy],
        episode: Episode,
        env_index: int,
        **kwargs,
    ):


        portfolio_valuation = episode.hist_data["portfolio_values"][-1]
        position_changes = np.sum(np.diff(episode.hist_data["positions"]) != 0)
        # m_port=np.mean(position_changes)
        
        episode.custom_metrics["portfolio_value"] = float(portfolio_valuation)
        episode.custom_metrics["position_changes"] = float(position_changes)
        # train.report(episode.custom_metrics)
        


In [ ]:
search_space={
    'lr':tune.loguniform(1e-6, 1e-1),
                         'gamma':tune.loguniform(.9, .999),      
                        #  'kl_coeff':tune.uniform(.001,.5),
                        #  num_sgd_iter=tune.randint(1,24),
                         'model':{
                             'fcnet_hiddens': tune.choice([[128,128],[256,256],[512,512],[1024,1024]]),
                            #  'use_attention':True,#tune.choice([True,False]),

                                },
}

In [ ]:

ppo_algo = (PPOConfig().training(lr=tune.loguniform(1e-8, 1e-5),
                         gamma=tune.loguniform(.8, .999),      
                         lambda_=tune.loguniform(.9, 1),
                         vf_loss_coeff=tune.loguniform(.5, 1),     
                         kl_coeff=tune.loguniform(.003,.3),
                         num_sgd_iter=tune.randint(3,30),
                         clip_param=tune.choice([.1,.2,.3]),
                         model={
                             'fcnet_hiddens': tune.choice([[128,128],[256,256],[512,512],[1024,1024]]),
                            #  'use_attention':True,#tune.choice([True,False]),

                                },

                         )
            # .rollouts(num_rollout_workers=4)
            .resources(
                        num_gpus=1
                        )
            .callbacks(TraderCallbacks)
            .reporting(keep_per_episode_custom_metrics=False)
            .environment(env="train_env")
            .debugging(log_level='ERROR')
                
            .evaluation(
            #             # custom_evaluation_function=custom_eval_function,
            #             evaluation_interval=100,
            #             evaluation_duration=10,
            #             evaluation_duration_unit='episode',
            #             evaluation_parallel_to_training=True,
            #             evaluation_num_workers=1,
            #             always_attach_evaluation_results=True,

                        evaluation_config={

                                            "env": 'test_env',  # The dictionary we built before (only the overriding keys to use in evaluation)
                                            "explore": False,
                                            }
                        )
            
            
            )

ppo_algo_config=ppo_algo.to_dict()
# ppo_algo_config

In [ ]:
# ppo_algo.build()

In [ ]:
# First, get the handle for the logger you want to modify
ray_data_logger = logging.getLogger("ray.data")
ray_tune_logger = logging.getLogger("ray.tune")
ray_rllib_logger = logging.getLogger("ray.rllib")
ray_train_logger = logging.getLogger("ray.train")
ray_serve_logger = logging.getLogger("ray.serve")


ray_data_logger .setLevel(logging.ERROR)
ray_tune_logger .setLevel(logging.ERROR)
ray_rllib_logger.setLevel(logging.ERROR)
ray_train_logger.setLevel(logging.ERROR)
ray_serve_logger.setLevel(logging.ERROR)

In [ ]:
logger = logging.getLogger("ray")
logger.setLevel(logging.ERROR)

In [ ]:
ray.init(include_dashboard=True,
         
         _temp_dir='C:/Users/standard/Git/MultiTrader/Rayrl_tune/tmp/',
         dashboard_host="0.0.0.0")

In [ ]:
checkpoint_path='/Rayrl_tune/checkpoints/'

checkpoint_config=train.CheckpointConfig(checkpoint_at_end=True,
                                        #  checkpoint_score_attribute='custom_metrics/portfolio_value_mean',
                                         checkpoint_score_attribute='episode_reward_mean',
                                         num_to_keep=1,
                                        #  checkpoint_frequency=10,
                                         )
reporter= JupyterNotebookReporter(
    sort_by_metric=True,
    metric='custom_metrics/portfolio_value_max',
    mode='max',


    metric_columns={
                    'training_iteration':'iter',
                    'reward':'reward',
                    'custom_metrics/portfolio_value_max':'portfolio_value_max',
                    'custom_metrics/position_changes_mean': 'position_changes_max',

                    'episode_reward_mean':'episode_reward_mean',
                    'custom_metrics/portfolio_value_mean': 'portfolio_value_mean',
                    'custom_metrics/position_changes_mean': 'position_changes_mean',
                    'custom_metrics/position_changes': 'position_changes',
                    
                    'episode_reward_max': 'episode_reward_max',
                    'episode_reward_min':'episode_reward_min',
                    'custom_metrics/portfolio_value_min': 'portfolio_value_min',
                    

    },
    parameter_columns={

                        'lr':'lr',
                        'gamma':'gamma',
                        'lambda_':'lambda',
                        'kl_coeff':'kl_coeff',
                        'vf_loss_coeff':'vf_loss_coeff',
                        'num_sgd_iter':'num_sgd_iter',
                        'model/fcnet_hiddens':'fcnet_hiddens',
                        # 'model/use_attention':'use_attention'
                        }
    )

run_config=train.RunConfig(

                            stop={
                                "training_iteration": 200,
                                #   "evaluation/episode_reward_mean": 1.5,
                                #   "timesteps_total":5000,
                                },
                                
                            
                            storage_path=checkpoint_path,
                            checkpoint_config=checkpoint_config,
                            progress_reporter=reporter

                        )
# search_algo = OptunaSearch(
#                             mode="max",                            
#                             metric='custom_metrics/portfolio_value_mean',
# )
# search_algo= ConcurrencyLimiter(
#                                 search_algo, 
#                                 max_concurrent=1
#                                 )


search_algo = HyperOptSearch(metric='custom_metrics/portfolio_value_mean',
                             mode='max',
                             )


tune_config=tune.TuneConfig(num_samples=10,
                            reuse_actors=False,
                            search_alg=search_algo,
                            trial_name_creator=trial_str_creator,
                            trial_dirname_creator=trial_str_creator,

                            )
css = HTML('<link rel="stylesheet" type="text/css" href="./custom.css">') # make sure the path is correct, it fails silently otherwise
display(css)
tuner = tune.Tuner(
    "PPO",
    param_space=ppo_algo_config,
    tune_config=tune_config,
    run_config=run_config,
    )
appo_results=tuner.fit()

In [ ]:


appo_results=tuner.fit()

In [ ]:
df_results = appo_results.get_dataframe()
df_results.to_csv('Rayrl_tune/results/appo_tune.csv')
# df_results

In [ ]:

custom_cols=[
                'config/lr',
                'config/gamma',
                'config/lambda',
                'config/kl_coeff',
                'config/model/fcnet_hiddens',

                ]
custom_cols+=[c for c in df_results.filter(like='episode_reward').columns]
custom_cols+=[c for c in df_results.filter(like='custom_metrics').columns]
custom_cols=[c for c in custom_cols if 'sampler_results' not in c ]
custom_cols=[c for c in custom_cols if 'hist_stats' not in c ]
custom_cols=list(set(custom_cols))

In [ ]:

df_results[custom_cols].sort_values('custom_metrics/portfolio_value_mean',ascending=False)


In [ ]:
# [c for c in df_results.columns]

In [ ]:
best=appo_results.get_best_result(metric='custom_metrics/portfolio_value_mean', mode='max')
best.checkpoint

In [ ]:
from ray.rllib.algorithms.algorithm import Algorithm

In [ ]:
agent= Algorithm.from_checkpoint(best.checkpoint.path)

In [ ]:
num_epochs=1000
for i in tqdm(range(num_epochs)):
    agent.train()
    if i % 100 == 0:
        checkpoint_dir = agent.save('C:/Users/standard/Git/MultiTrader/Rayrl_tune/checkpoints').checkpoint.path
        print(f"Checkpoint saved in directory {checkpoint_dir}")

In [ ]:
test_env=NeuralForecastingTradingEnv(**test_env_config)

In [ ]:
obs,info=train_env.reset()
action=None
reward=0
terminated = truncated = False

while not terminated and not truncated:

    action = agent.compute_single_action(obs,explore=False)
    action = train_env.action_space.sample()
    # print(action)
    next_obs, reward, terminated, truncated, info = train_env.step(action)

    obs=next_obs
info
train_env.get_metrics(),info['portfolio_valuation']

In [ ]:
eval_profit=[]
for i in tqdm( range(100)):
    obs,info=train_env.reset()
    action=None
    reward=0
    terminated = truncated = False

    while not terminated and not truncated:

        action = agent.compute_single_action(obs,explore=False)
        # action = train_env.action_space.sample()
        # print(action)
        next_obs, reward, terminated, truncated, info = train_env.step(action)

        obs=next_obs

    eval_profit.append(info['portfolio_valuation'])
    info['portfolio_valuation']
profits=[e for e in eval_profit]
plt.plot(profits)
np.mean(profits)

In [ ]:
eval_profit=[]
for i in tqdm( range(100)):
    obs,info=test_env.reset()
    action=None
    reward=0
    terminated = truncated = False

    while not terminated and not truncated:

        action = agent.compute_single_action(obs,explore=False)
        # action = train_env.action_space.sample()
        # print(action)
        next_obs, reward, terminated, truncated, info = test_env.step(action)

        obs=next_obs

    eval_profit.append(info['portfolio_valuation'])
    info['portfolio_valuation']


In [ ]:

plt.plot(eval_profit)
np.mean(eval_profit)

In [ ]:
obs,info=test_env.reset()
action=None
reward=0
terminated = truncated = False

while not terminated and not truncated:

    action = agent.compute_single_action(obs,explore=False)
    # action = train_env.action_space.sample()
    # print(action)
    next_obs, reward, terminated, truncated, info = test_env.step(action)

    obs=next_obs


info['portfolio_valuation']

In [ ]:
test_env.save_for_render(dir = "PPO_test_render_logs")

In [ ]:
renderer = Renderer(render_logs_dir="render_logs")

In [ ]:
# Add Custom Metrics (Annualized metrics)
renderer.add_metric(
    name = "Annual Market Return",
    function = lambda df : f"{ ((df['close'].iloc[-1] / df['close'].iloc[0])**(pd.Timedelta(days=365)/(df.index.values[-1] - df.index.values[0]))-1)*100:0.2f}%"
)
renderer.add_metric(
        name = "Annual Portfolio Return",
        function = lambda df : f"{((df['portfolio_valuation'].iloc[-1] / df['portfolio_valuation'].iloc[0])**(pd.Timedelta(days=365)/(df.index.values[-1] - df.index.values[0]))-1)*100:0.2f}%"
)

renderer.run()